<a href="https://colab.research.google.com/github/duvancatan/Time_Series_UdeA/blob/main/2_Simulacion_ARMA_ACF_PACF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación Distribuciones Muestrales de ACF y PACF, y Modelos ARMA(p,q)

Esta simulación tiene como objetivo ilustrar empíricamente que, para una serie estacionaria como el ruido blanco, los estimadores muestrales de la función de autocorrelación (ACF) y de autocorrelación parcial (PACF) en rezagos fijos siguen aproximadamente una distribución normal

$$\hat{\rho}(h),\ \hat{\phi}_{hh} \sim \mathcal{N} \left( 0, \frac{1}{n} \right)$$

cuando el tamaño de muestra n es suficientemente grande.


Adicionalmente, simulamos de modelos autorregresivos de media móvil (ARMA) es una herramienta fundamental en el análisis de Series de Tiempo. Los modelos ARMA(p,q) son una combinación de procesos autorregresivos (AR) y de media móvil (MA), donde "p" representa el orden del componente AR y "q" representa el orden del componente MA.

En la simulación de un modelo ARMA(p,q), se generan datos siguiendo un proceso estocástico definido por una combinación lineal de los términos autorregresivos y de media móvil, los cuales son ampliamente utilizados en finanzas, economía, ingeniería, entre otros campos, para modelar y predecir valores futuros.

La simulación de un modelo ARMA implica la generación de una secuencia de datos que reflejan el comportamiento estocástico de la serie temporal bajo consideración, siguiendo las especificaciones del modelo ARMA(p,q) dado. Este proceso de simulación es esencial para comprender el comportamiento del modelo, evaluar su validez y desempeño, así como para realizar análisis de sensibilidad y pruebas de hipótesis.

 En R el proceso ARMA(p,q) con media cero esta definido por:

 $$x_{t} = a_1x_{t-1} + \ldots + a_p x_{t-p} + \omega_t + b_1\omega_{t-1} + \ldots + b_q\omega_{t-q}$$









# Librerías




In [ ]:
# Instalar Librerías #
install.packages("stats")
install.packages("astsa")
install.packages("forecast")
install.packages("TSA")

In [ ]:
# Cargar Librerías #
library(stats)
library(astsa)
library(forecast)
library("TSA")

# 1. Distribución Empírica de la ACF y PACF

Se realiza una simulación Monte Carlo para analizar la distribución muestral de los estimadores de autocorrelación y autocorrelación parcial en series estacionarias. Los resultados confirman que, en rezagos fijos, estos estimadores se aproximan a una distribución normal con media cero y varianza $1/n.$

In [ ]:
# Definición de Parámetros
B <- 1000                   # Número de simulaciones
n_values <- c(50, 100, 200, 500, 1000) # Tamaños muestrales
max_lag <- 10               # Máximo rezago

In [ ]:
# Simulación de Ruido Blanco Normal con sus ACF y PACF
for (n in n_values) {
  acf_mat <- matrix(NA, nrow = B, ncol = max_lag)
  pacf_mat <- matrix(NA, nrow = B, ncol = max_lag)

  for (b in 1:B) {
    x <- rnorm(n)  # Serie de ruido blanco
    acf_vals <- acf(x, lag.max = max_lag, plot = FALSE)$acf[-1]   # quitar lag 0
    pacf_vals <- pacf(x, lag.max = max_lag, plot = FALSE)$acf

    acf_mat[b, ] <- acf_vals
    pacf_mat[b, ] <- pacf_vals
  }

  # Analizar un rezago fijo (por ejemplo h = 1)
  h <- 1
  acf_h <- acf_mat[, h]
  pacf_h <- pacf_mat[, h]

  var_teo <- 1 / n  # Varianza teórica

  # Graficar histogramas
  par(mfrow = c(1, 2))  # Dos gráficos lado a lado

  hist(acf_h, probability = TRUE, col = "lightblue", main = paste("ACF en h =", h, " (n =", n, ")"),
       xlab = "Valor de ACF", xlim = c(-0.3, 0.3), breaks = 20)
  curve(dnorm(x, mean = 0, sd = sqrt(var_teo)), col = "red", lwd = 2, add = TRUE)

  hist(pacf_h, probability = TRUE, col = "gray", main = paste("PACF en h =", h, " (n =", n, ")"),
       xlab = "Valor de PACF", xlim = c(-0.3, 0.3), breaks = 20)
  curve(dnorm(x, mean = 0, sd = sqrt(var_teo)), col = "blue", lwd = 2, add = TRUE)

  # Mostrar medias y varianzas empíricas
  cat("\nResultados para n =", n, "y rezago h =", h,",", "Varianza Teórica=",var_teo, "\n")
  cat("Media ACF:  ", round(mean(acf_h), 4), "   Varianza ACF:  ", round(var(acf_h), 4), "\n")
  cat("Media PACF: ", round(mean(pacf_h), 4), "   Varianza PACF: ", round(var(pacf_h), 4), "\n")
}

# 2. Procesos Autorregresivos

## AR(1)

In [ ]:
# Número de observaciones a simular #
num_sim = 500

In [ ]:
# Definición de los parametros del proceso #
theta0_ar1 = 5 # Media
phi1_ar1 = 0.8 # Parámetro autorregresivo
var_RB_ar1 = 2 # Varianza del Ruido Blanco

In [ ]:
# Simulación de un proceso AR(1) con at Normal #
z_ar1 = arima.sim(n = num_sim, list(ar = phi1_ar1), mean = theta0_ar1, sd = sqrt(var_RB_ar1), n.start = 50)

In [ ]:
# Gráfico de la serie simulada #
options(repr.plot.width=14, repr.plot.height=8)
plot.ts(z_ar1, main = "Simulación de Proceso AR(1)");grid()

In [ ]:
# Función para graficar el círculo unitario
plot_unit_circle <- function() {
  theta <- seq(0, 2 * pi, length.out = 100)
  x <- cos(theta)
  y <- sin(theta)
  lines(x, y, type = "l", col = "black", lty = 2)
}

In [ ]:
# Calcular las raíces del polinomio característico #
phi1 <- phi1_ar1  # Coeficiente de MA(1)
polinomio <- c(1, -phi1) # Coeficientes del polinomio característico
raices <- polyroot(polinomio) # Calcular las raíces

In [ ]:
# Graficar las raíces en el círculo unitario junto con el círculo unitario #
plot(1, type="n", xlim=c(-2.0, 2.0), ylim=c(-2.0, 2.0), xlab="", ylab="", main="Raíces en el círculo unitario")
plot_unit_circle() # Graficar el círculo unitario
symbols(raices, rep(0, length(raices)), circles=rep(1, length(raices)), inches=0.05, add=TRUE)
abline(h=0, v=0) # Agregar ejes

In [ ]:
# ACF y PACF individuales #
par(mfrow=c(2,1))
acf(z_ar1, main = "ACF de Proceso AR(1)", ci=0)
pacf(z_ar1, lag.max=15, , main = "PACF de Proceso AR(1)")

In [ ]:
# Obtención de la ACF y PACF, presentacion en un solo gráfico y sus valores #
acf2(z_ar1, main = "ACF y PACF de Proceso AR(1)") # Graficar simultaneamente ACF y PACF

## Estimación AR(1)

In [ ]:
# Función ar() #
?ar

In [ ]:
# Selecciona orden y Estimación #
mod_ar10 <- ar(z_ar1)
(mod_ar10)

In [ ]:
# Salidas de la función #
names(mod_ar10)

In [ ]:
# Media estimada #
med1 <- mod_ar10$x.mean
cat('Media Estimada:', med1)

In [ ]:
var1 <- mod_ar10$asy.var.coef # Var Asint Estimada de Parámetros
cat('Varianza Estimada:', var1)

In [ ]:
sd1 <- sqrt(var1) # DS Asint Estimada de Parámetros
cat('Desviación Estándar Estimada:', sd1)

In [ ]:
# Función arima() #
?arima

In [ ]:
# Parámetros Estimados #
mod_ar11 <- arima(z_ar1, order = c(1,0,0))
(mod_ar11)

In [ ]:
# Salidas de la función #
names(mod_ar11)

In [ ]:
# Función sarima() #
?sarima

In [ ]:
# Estimación y diagnótico con SARIMA() #
mod_ar12<-sarima(z_ar1, p=1,d=0,q=0);
(mod_ar12)

In [ ]:
# Información general del modelo #
summary(mod_ar12)

In [ ]:
# Modelo usando la función auto.arima() #
mod_aut<-auto.arima(z_ar1)
mod_aut

## AR(2)

In [ ]:
# Número de observaciones a simular #
num_sim = 500

In [ ]:
# Definición de los parametros del proceso #
theta0_ar2 = 3 # Intercepto
phi1_ar2 = 1.2 # Parametro autorregresivo 1
phi2_ar2 = -0.32 # Parametro autorregresivo 2
var_RB_ar2 = 1 # Varianza del RB

In [ ]:
# Simulacion de un proceso AR(2) con at Normal #
z_ar2 = arima.sim(n = num_sim, list(ar = c(phi1_ar2, phi2_ar2)), mean = theta0_ar2, sd = sqrt(var_RB_ar2), n.start = 20)

In [ ]:
# Gráfico de la serie simulada #
ts.plot(z_ar2, main = "Simulación de Proceso AR(2)") ;grid()

In [ ]:
# Función para graficar el círculo unitario
plot_unit_circle <- function() {
  theta <- seq(0, 2 * pi, length.out = 100)
  x <- cos(theta)
  y <- sin(theta)
  lines(x, y, type = "l", col = "black", lty = 2)
}

In [ ]:
# Calcular las raíces del polinomio característico #
phi1 <- phi1_ar2  # Coeficiente de AR(1)
phi2 <- phi2_ar2 # Coeficiente de AR(2)
polinomio <- c(1, -phi1, -phi2) # Coeficientes del polinomio característico
raices <- polyroot(polinomio) # Calcular las raíces

In [ ]:
# Graficar las raíces en el círculo unitario junto con el círculo unitario #
plot(1, type="n", xlim=c(-2.7, 2.7), ylim=c(-2.7, 2.7), xlab="", ylab="", main="Raíces en el círculo unitario")
plot_unit_circle() # Graficar el círculo unitario
symbols(raices, rep(0, length(raices)), circles=rep(1, length(raices)), inches=0.05, add=TRUE)
abline(h=0, v=0) # Agregar ejes

In [ ]:
# ACF y PACF individuales #
par(mfrow=c(2,1))
acf(z_ar2, main = "ACF de Proceso AR(2)", ci=0)
pacf(z_ar2, lag.max=15, , main = "PACF de Proceso AR(2)")

In [ ]:
# Obtención de la ACF y PACF, presentacion en un solo gráfico y sus valores #
acf2(z_ar2, main = "ACF y PACF de Proceso AR(2)") # Graficar simultaneamente ACF y PACF

## Estimación AR(2)

In [ ]:
# Función ar() #
?ar

In [ ]:
# Selecciona orden y Estimación #
mod_ar20 <- ar(z_ar2)
(mod_ar20)

In [ ]:
# Salidas de la función #
names(mod_ar20)

In [ ]:
# Media estimada #
med2 <- mod_ar20$x.mean
cat('Media Estimada:', med2)

In [ ]:
var2 <- mod_ar20$asy.var.coef # Var Asint Estimada de Parámetros
cat('Varianza Estimada:', var2)

In [ ]:
sd2 <- sqrt(var2) # DS Asint Estimada de Parámetros
cat('Desviación Estándar Estimada:', sd2)

In [ ]:
# Función arima() #
?arima

In [ ]:
# Parámetros Estimados
mod_ar21 <- arima(z_ar2, order = c(2,0,0))
(mod_ar21)

In [ ]:
# Salidas de la función #
names(mod_ar21)

In [ ]:
# Función sarima() #
?sarima

In [ ]:
# Estimación y diagnótico con SARIMA() #
mod_ar22<-sarima(z_ar2, p=2,d=0,q=0);
(mod_ar22)

In [ ]:
# Información general del modelo #
summary(mod_ar22)

In [ ]:
# Modelo usando la función auto.arima() #
mod_aut<-auto.arima(z_ar2)
mod_aut

# 3. Procesos de Medias Móviles

## MA(1)

In [ ]:
# Número de observaciones a simular #
num_sim = 500

In [ ]:
# Definicion de los parametros del proceso #
theta0_ma1 = 5
theta1_ma1 = 0.7
var_RB_ma1 = 2

In [ ]:
# Simulacion de un proceso MA(1) Normal #
z_ma1 = theta0_ma1 + arima.sim(n = num_sim, list(ma = theta1_ma1), sd = sqrt(var_RB_ma1), n.start=50)

In [ ]:
# Gráfico de la serie simulada #
ts.plot(z_ma1, main = "Simulación de Proceso MA(1)") ;grid()

In [ ]:
# Función para graficar el círculo unitario
plot_unit_circle <- function() {
  theta <- seq(0, 2 * pi, length.out = 100)
  x <- cos(theta)
  y <- sin(theta)
  lines(x, y, type = "l", col = "black", lty = 2)
}

In [ ]:
# Calcular las raíces del polinomio característico #
theta1 <- theta1_ma1  # Coeficiente de MA(1)
polinomio <- c(1, -theta1) # Coeficientes del polinomio característico
raices <- polyroot(polinomio) # Calcular las raíces
raices

In [ ]:
# Graficar las raíces en el círculo unitario junto con el círculo unitario #
plot(1, type="n", xlim=c(-2.0, 2.0), ylim=c(-2.0, 2.0), xlab="", ylab="", main="Raíces en el círculo unitario")
plot_unit_circle() # Graficar el círculo unitario
symbols(raices, rep(0, length(raices)), circles=rep(1, length(raices)), inches=0.05, add=TRUE)
abline(h=0, v=0) # Agregar ejes

In [ ]:
# ACF y PACF individuales #
par(mfrow=c(2,1))
acf(z_ma1, main = "ACF de Proceso MA(1)")
pacf(z_ma1, lag.max=15, , main = "PACF de Proceso MA(1)", ci=0)

In [ ]:
# Obtención de la ACF y PACF, presentacion en un solo gráfico y sus valores #
acf2(z_ma1, main = "ACF y PACF de Proceso MA(1)") # Graficar simultaneamente ACF y PACF

## Estimación MA(1)

In [ ]:
# Función arima() #
?arima

In [ ]:
# Parámetro Estiamdos #
mod_ma10 <- arima(z_ma1, order = c(0,0,1))
(mod_ma10)

In [ ]:
# Salidas de la función #
names(mod_ma10)

In [ ]:
# Función sarima() #
?sarima

In [ ]:
# Estimación y diagnótico con SARIMA() #
mod_ma12<-sarima(z_ma1, p=0,d=0,q=1);
(mod_ma12)

In [ ]:
# Información general del modelo #
summary(mod_ma12)

In [ ]:
# Modelo usando la función auto.arima() #
mod_aut<-auto.arima(z_ma1)
mod_aut

## MA(2)

In [ ]:
# Número de observaciones a simular #
num_sim = 500

In [ ]:
# Definicion de los parametros del proceso #
theta0_ma2 = 0
theta1_ma2 = 0.4
theta2_ma2 =-0.3
var_RB_ma2 = 2

In [ ]:
# Simulacion de un proceso MA(2) Normal #
z_ma2 = arima.sim(n = num_sim, list(ma = c(theta1_ma2, theta2_ma2)), sd = sqrt(var_RB_ma2), n.start=50)

In [ ]:
# Gráfico de la serie simulada #
ts.plot(z_ma2, main = "Simulación de Proceso MA(2)") ;grid()

In [ ]:
# Función para graficar el círculo unitario
plot_unit_circle <- function() {
  theta <- seq(0, 2 * pi, length.out = 100)
  x <- cos(theta)
  y <- sin(theta)
  lines(x, y, type = "l", col = "black", lty = 2)
}

In [ ]:
# Calcular las raíces del polinomio característico #
theta1 <- theta1_ma2  # Coeficiente de MA(1)
theta2 <- theta2_ma2  # Coeficiente de MA(2)
polinomio <- c(1, -theta1, -theta2) # Coeficientes del polinomio característico
raices <- polyroot(polinomio) # Calcular las raíces
(raices=abs(raices))

In [ ]:
# Graficar las raíces en el círculo unitario junto con el círculo unitario #
plot(1, type="n", xlim=c(-2.8, 2.8), ylim=c(-2.8, 2.8), xlab="", ylab="", main="Raíces en el círculo unitario")
plot_unit_circle() # Graficar el círculo unitario
symbols(raices, rep(0, length(raices)), circles=rep(1, length(raices)), inches=0.05, add=TRUE)
abline(h=0, v=0) # Agregar ejes

In [ ]:
# ACF y PACF individuales #
par(mfrow=c(2,1))
acf(z_ma2, main = "ACF de Proceso MA(2)")
pacf(z_ma2, lag.max=15, , main = "PACF de Proceso MA(2)", ci=0)

In [ ]:
# Obtención de la ACF y PACF, presentacion en un solo gráfico y sus valores #
acf2(z_ma2, main = "ACF y PACF de Proceso MA(2)") # Graficar simultaneamente ACF y PACF

## Estimación MA(2)

In [ ]:
# Función arima() #
?arima

In [ ]:
# Parámetros Estimados #
mod_ma21 <- arima(z_ma2, order = c(0,0,2), method = c("CSS-ML"))
(mod_ma21)

In [ ]:
# Salidas de la función #
names(mod_ma21)

In [ ]:
# Información general del modelo #
summary(mod_ma21)

In [ ]:
# Función sarima() #
?sarima

In [ ]:
# Estimación alternativa con SARIMA() #
mod_ma22<-sarima(z_ma2, p=0,d=0,q=2);
(mod_ma22)

In [ ]:
# Modelo usando la función auto.arima() #
mod_aut<-auto.arima(z_ma2)
mod_aut

# 4. Procesos Autorregresivos y de Medias Móviles

## ARMA(1,1)

In [ ]:
# Número de observaciones a simular #
num_sim = 500

In [ ]:
# Definicion de los parametros del proceso #
theta0_arma11 = 0 # Intercepto
phi1_arma11 = 0.5 # Parámetro AR
theta1_arma11 = 0.7 # Parámetro MA
var_RB_arma11 = 2 # Varianza Ruido Blanco

In [ ]:
# Simulacion de un proceso ARMA(1,1) con at N(0,2) #
z_arma11 = arima.sim(n = num_sim, list(ar = phi1_arma11, ma = theta1_arma11), mean = theta0_arma11,  sd = sqrt(var_RB_arma11), n.start=50)

In [ ]:
# Gráfico de la serie simulada #
ts.plot(z_arma11, main = "Simulación de Proceso ARMA(1,1)") ;grid()

In [ ]:
# ACF y PACF individuales #
par(mfrow=c(2,1))
acf(z_arma11, main = "ACF de Proceso ARMA(1,1)")
pacf(z_arma11, lag.max=15, , main = "PACF de Proceso ARMA(1,1)")

In [ ]:
# Obtención de la ACF y PACF, presentacion en un solo gráfico y sus valores #
acf2(z_arma11, main = "ACF y PACF de Proceso ARMA(1,1)") # Graficar simultaneamente ACF y PACF

In [ ]:
# Función de Autocorrelación Extendida EACF #
eacf(z_arma11)

## Estimación ARMA(1,1)

In [ ]:
# Función arima() #
?arima

In [ ]:
# Parámetro Estimados #
mod_arma1 <- arima(z_arma11, order = c(1,0,1))
(mod_arma1)

In [ ]:
# Salidas de la función #
names(mod_arma1)

In [ ]:
# Información general del modelo #
summary(mod_arma1)

In [ ]:
# Función sarima() #
?sarima

In [ ]:
# Estimación alternativa con SARIMA() #
mod_arma2<-sarima(z_arma11, p=1,d=0,q=1);

In [ ]:
# Modelo usando la función auto.arima() #
mod_aut<-auto.arima(z_arma11)
mod_aut

# 5. Representación MA(infty) de un ARMA(p,q)

In [ ]:
# Función ARMAtoMA() #
?ARMAtoMA

In [ ]:
# 10 coeficientes de un AR(1) #
ARMAtoMA(ar=0.7, ma=-0.4, lag.max=10)

In [ ]:
# 10 coeficientes de un ARMA(1,1) y 1.5 es el coeficiente del término de media móvil (MA) #
ARMAtoMA(c(1.0, -0.25), 1.5, lag.max=10)

In [ ]:
# 10 coeficientes de un ARMA(1,1) #
ARMAtoMA(c(1.0, -0.25), lag.max=10)